In [1]:
import os

In [2]:
%pwd

'/home/smruti/projects/skin_disease_classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/smruti/projects/skin_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from skin_disease_classifier.constants import *
from skin_disease_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2025-03-08 09:21:16.370175: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 09:21:16.415294: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import cv2
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        data_dir = self.config.training_data
        seed = 123

        # Splitting data and creating training and validation datasets
        self.train_ds = tf.keras.utils.image_dataset_from_directory(
            data_dir,
            validation_split=0.1,
            subset="training",
            seed=seed,
            image_size=(self.config.params_image_size[0], self.config.params_image_size[1]),
            batch_size=self.config.params_batch_size
        )

        self.validation_ds = tf.keras.utils.image_dataset_from_directory(
            data_dir,
            validation_split=0.1,
            subset="validation",
            seed=seed,
            image_size=(self.config.params_image_size[0], self.config.params_image_size[1]),
            batch_size=self.config.params_batch_size
        )

        # Normalizing Pixel Values for Training and Validation Datasets
        self.train_ds = self.train_ds.map(lambda x, y: (x / 255.0, y))
        self.validation_ds = self.validation_ds.map(lambda x, y: (x / 255.0, y))

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        early_stopping = EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            verbose=1,
            mode='max',
            restore_best_weights=True,
            baseline=0.98
        )

        self.model.fit(
            self.train_ds,
            epochs=self.config.params_epochs,
            validation_data=self.validation_ds,
            callbacks=[early_stopping]
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-03-08 09:21:18,429: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-08 09:21:18,431: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-08 09:21:18,432: INFO: common: created directory at: artifacts]
[2025-03-08 09:21:18,434: INFO: common: created directory at: artifacts/training]


2025-03-08 09:21:18.467231: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-08 09:21:18.472917: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-08 09:21:18.472953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-08 09:21:18.476805: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-08 09:21:18.476839: I external/local_xla/xla/stream_executor

Found 9548 files belonging to 5 classes.
Using 8594 files for training.
Found 9548 files belonging to 5 classes.
Using 954 files for validation.
Epoch 1/100


I0000 00:00:1741425690.053562   23001 service.cc:145] XLA service 0x7f22cc002970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741425690.053629   23001 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-03-08 09:21:30.536461: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-08 09:21:32.646749: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2025-03-08 09:21:40.219578: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng55{k2=0,k13=2,k14=2,k18=0,k22=0,k23=1} for conv (f32[32,128,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,192,56,56]{3,2,1,0}, f32[128,192,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_i

  1/269 ━━━━━━━━━━━━━━━━━━━━ 2:27:05 33s/step - accuracy: 0.2188 - loss: 2.2955

I0000 00:00:1741425715.561711   23001 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


269/269 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.7397 - loss: 0.7272

2025-03-08 09:22:48.675900: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng48{k2=2,k6=2,k13=2,k14=0,k22=1} for conv (f32[26,128,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[26,224,56,56]{3,2,1,0}, f32[128,224,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2025-03-08 09:22:48.685044: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 2.062583988s
Trying algorithm eng48{k2=2,k6=2,k13=2,k14=0,k22=1} for conv (f32[26,128,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[26,224,56,56]{3,2,1,0}, f32[128,224,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_que

269/269 ━━━━━━━━━━━━━━━━━━━━ 100s 249ms/step - accuracy: 0.7400 - loss: 0.7263 - val_accuracy: 0.9109 - val_loss: 0.2451
Epoch 2/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 22s 83ms/step - accuracy: 0.9042 - loss: 0.2604 - val_accuracy: 0.9560 - val_loss: 0.1374
Epoch 3/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 19s 71ms/step - accuracy: 0.9264 - loss: 0.2068 - val_accuracy: 0.9539 - val_loss: 0.1188
Epoch 4/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 23s 85ms/step - accuracy: 0.9382 - loss: 0.1654 - val_accuracy: 0.9560 - val_loss: 0.1172
Epoch 5/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9412 - loss: 0.1569 - val_accuracy: 0.9696 - val_loss: 0.0845
Epoch 6/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 22s 83ms/step - accuracy: 0.9588 - loss: 0.1164 - val_accuracy: 0.9801 - val_loss: 0.0735
Epoch 7/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 23s 87ms/step - accuracy: 0.9592 - loss: 0.1141 - val_accuracy: 0.9717 - val_loss: 0.0874
Epoch 8/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - accuracy: 0.9634 - loss: 0.1009 - va